---
<center>

  # **Tarea 08: Efecto Stark**

**Realizado Por:**

   Samuel Huertas Rojas

---
</center>

# Efecto Stark
A partir de los elementos de matriz calcule el efectoStark para los estados $n=2$ y $n=3$ del átomo de hidrógeno en función del campo $E$

Tenemos que para experimentar el efecto Stark es necesario tener un campo electrico del orden de los $10^{7} [V/m]$, vemos que para los estados no degenerados del átomo de hidrogeno no se va a poder tener momento dipolar permanente pues habria cambio de energía, lo cual no puede ocurrir. Por lo que el estudio del efecto Star se debe de realizar en estados degenerados, como lo son el $n=2$ y $n=3$.

Vemos que la energía va a ser:

$$
E_{n=2} = - mc^2 \dfrac{(Z \alpha)^2}{8}
$$

Tenemos que el Hamiltoniano no perturbado va a ser:

$$
\hat{H}_0 = -\dfrac{\hbar^2}{2 m} \nabla^2 - \dfrac{Z e^2}{4 \pi \epsilon_0 r}
$$

Tenemos que los elemntos de matriz que no se van a anular son $$ \{\ket{1,1}, \ket{1,0}, \ket{1,-1}, \ket{0, 0}\}  $$

Se tiene: 

$$
\begin{bmatrix}
 -E_1^{(1)} & H'_{12} \\
 H'_{21} & -E_2^{(1)}
\end{bmatrix}
\begin{bmatrix}
 C_1 \\
 C_2
\end{bmatrix}
= 0
$$

Con $H'_{12} = H'_{21} = e E z = e E \braket{\psi_{2,1,0} | r \cos \theta| \psi_{2,0,0}} = - 3 e E \dfrac{a_0}{Z}$

In [ ]:
import sympy as sp
import numpy as np

from sympy.physics.quantum.state import OrthogonalKet
from sympy.physics.hydrogen import Psi_nlm, E_nl

Ket = OrthogonalKet


In [ ]:
# Constantes
alpha = 1 / 137
c = 1 / alpha  # Velocidad de la luz en unidades atomicas
mu = 1  # Masa reducida en unidades atomicas
me = 1  # Masa del electron en unidades atomicas
e = 1  # Carga del electron en unidades atomicas
Z = 1  # Numero atomico de lHidorgeno
a0 = 1  # Radio de Bohr en unidades atomicas

# Valor del campo electrico
E = 1

# Definicion de los simbolos
r, theta, phi = sp.symbols("r theta phi", real=True)

hbar = sp.symbols("hbar", real=True)

# Definición de la variable z
z = r * sp.cos(theta)

# Definición de los factores de converción hartree al sistema internacional
factor_convercion = {
    "joul": 4.3598e-18,
    "cm": 219474.6313,
    "eV": 27.2114,
    "tesla": 235051.755,
}

In [ ]:
# Definición de las bases a utilizar
def bases(n):
    bases = []

    for l in range(n):  # noqa: E741
        # Construccion de los kets
        elemento_base = [Ket(l, ml) for ml in np.arange(l, -l - 1, -1)]
        bases += elemento_base

    return bases


bases(2)

[|00>, |11>, |10>, |1-1>]

In [16]:
# Definición del Hamiltoniano perturbativo
def H1_elem(n, elem1, elem2):  # noqa: E741
    """Hamiltoniano del hidrogeno en unidades atomicas"""
    l1, ml1 = elem1.label
    l2, ml2 = elem2.label

    H1 = lambda F: e * E * z * F  # noqa: E731

    Psi_conjugate = sp.conjugate(Psi_nlm(n, l1, ml1, r, phi, theta))
    H_Psi = H1(Psi_nlm(n, l2, ml2, r, phi, theta))

    return sp.integrate(
        Psi_conjugate * H_Psi * r**2 * sp.sin(theta),
        (r, 0, sp.oo),
        (theta, 0, sp.pi),
        (phi, 0, 2 * sp.pi),
    )


# Definicion del Hamiltoniano sin perturbar
def H0(n):  # noqa: E741
    return E_nl(n)


def generar_matrices(n, estados):
    H1_matriz = [[0 for i in range(len(estados))] for j in range(len(estados))]
    H0_matriz = [[0 for i in range(len(estados))] for j in range(len(estados))]

    for i in range(len(estados)):
        for j in range(len(estados)):
            H1_matriz[i][j] = H1_elem(n, estados[i].dual, estados[j])
            if i == j:
                H0_matriz[i][j] = E_nl(n)

    return sp.Matrix(H1_matriz), sp.Matrix(H0_matriz)


n = 2
base = bases(n)

matriz1, matriz0 = generar_matrices(n, base)
matriz_total = matriz0 + matriz1
matriz_total

Matrix([
[-1/8,    0,   -3,    0],
[   0, -1/8,    0,    0],
[  -3,    0, -1/8,    0],
[   0,    0,    0, -1/8]])

In [17]:
# Diagonalizar la matriz
U, D = matriz_total.diagonalize()
D - matriz0

Matrix([
[-3, 0, 0, 0],
[ 0, 0, 0, 0],
[ 0, 0, 0, 0],
[ 0, 0, 0, 3]])

In [ ]:
# Obtenemos los auto estados
autoestado1 = U.col(0) / U.col(0).norm()
autoestado2 = U.col(3) / U.col(3).norm()

In [19]:
autoestado1

Matrix([
[sqrt(2)/2],
[        0],
[sqrt(2)/2],
[        0]])

In [20]:
# Asociación de los autoestados con los autovalores
def autoestado_autovalor(column, base):
    representacion = 0
    for i in range(len(base)):
        representacion += column[i] * base[i]
    return representacion

In [ ]:
print("Autoestado asociado ")
autoestado_autovalor(autoestado1, base)

sqrt(2)*|00>/2 + sqrt(2)*|10>/2

In [23]:
autoestado_autovalor(autoestado2, base)

-sqrt(2)*|00>/2 + sqrt(2)*|10>/2

In [28]:
# Para n = 3
n = 3
base = bases(n)

matriz1, matriz0 = generar_matrices(n, base)
matriz_total = matriz0 + matriz1

# Diagonalizar la matriz
U, D = matriz_total.diagonalize()
autovalores = D - matriz0

In [36]:
autovalores.cols
diagonal = [autovalores[i, i] for i in range(autovalores.rows)]
diagonal

[-9, -9/2, -9/2, 0, 0, 0, 9/2, 9/2, 9]

In [ ]:
lista_autoestados = []

for i in range(len(base)):
    autoestado = U.col(i) / U.col(i).norm()
    lista_autoestados.append(autoestado_autovalor(autoestado, base))


sqrt(3)*|00>/3 + sqrt(2)*|10>/2 + sqrt(6)*|20>/6

In [40]:
from collections import Counter

c = Counter(diagonal)

for i in c.items():
    print(f"El autovalor {i[0]} tiene una degeneracion de {i[1]}")

El autovalor -9 tiene una degeneracion de 1
El autovalor -9/2 tiene una degeneracion de 2
El autovalor 0 tiene una degeneracion de 3
El autovalor 9/2 tiene una degeneracion de 2
El autovalor 9 tiene una degeneracion de 1


In [45]:
# La representacion de los autoestados asociados a cada autovalor es la siguiente:
i = 6
print(f"Autoestado asociado al autovalor {diagonal[i]}:")
lista_autoestados[i]

Autoestado asociado al autovalor 9/2:


-sqrt(2)*|11>/2 + sqrt(2)*|21>/2